In [1]:
#Import some libraries
import numpy as np
import pandas as pd
# from thermoModel import *
from generateThermocline import generateThermocline
from maxDepth import maxDepth
from julianToNormal import jd_to_date
import matplotlib.pyplot as plt
from Net import Net
from global_land_mask import globe
import os
import xarray
import torch
import geopandas as gpd
import geodatasets

In [57]:
# setup global variables
latRange = [-90,90]                # range of world latitudes (degrees)
longRange = [-180,180]             # range of world longitudes (degrees)
dateRange = [2455562.5, 2455927.5]  # range of dates (julian time/days)

areaIncr = 20        # world area grid (degrees)
depthIncr = 1       # depth increment (meter)
dateIncr = 300        # date increment (days)
tempCutoff = 1      # thermocline temperature cutoff (K or degC)

latArray = np.arange(latRange[0], latRange[1], areaIncr)
longArray = np.arange(longRange[0], longRange[1], areaIncr)
dateArray = np.arange(dateRange[0], dateRange[1], dateIncr)

In [38]:
def generateThermoclineActual(df, lat, long, thermoActualPath):
    err = 0.1
    filtered_df = df.loc[(df['Latitude'].between(lat - err, lat + err)) 
            & (df['Longitude'].between(long - err, long + err)), ['Depth', 'Temperature']]
    # not enough data to filter by time as well
    sorted_df = filtered_df.sort_values(by=['Depth'])
    
    fig, ax = plt.subplots()
    ax.plot(sorted_df["Temperature"], sorted_df["Depth"])  
    ax.set_xlabel('Temperature ($^\circ$C)')
    ax.set_ylabel('Depth (m)')
    ax.set_title(f'Thermocline Actual for Lat: {lat:,.1f} Long: {long:,.1f}')
    ax.invert_yaxis()
    plt.savefig(f'{thermoActualPath}/ThermoclineActual_{lat:,.1f}_{long:,.1f}.png')
    plt.close('all')

    return sorted_df

In [25]:
def generateSurfaceTempActualMap(df, surfaceTempActualPath, date):
    plotDf = df.loc[(df['Depth'] == 0),['Longitude', 'Latitude', 'Temperature']]
    
    plotColor = "lightgrey"
    opacity = 0.5
    worldmap = gpd.read_file(geodatasets.get_path("naturalearth.land"))

    # Creating axes and plotting world map
    fig, ax = plt.subplots(figsize=(12, 6))
    worldmap.plot(color=plotColor, ax=ax)

    # Plotting our Impact Energy data with a color map
    x = plotDf['Longitude']
    y = plotDf['Latitude']
    z = plotDf['Temperature']
    plt.scatter(x, y, c=z, alpha=opacity, marker=".")
    plt.colorbar(label='Surface Temp ($^\circ$C)')
    plt.title("Global Surface Temperature Map")
    plt.xlabel("Longitude ($^\circ$)")
    plt.ylabel("Latitude ($^\circ$)")

    year,month,day = jd_to_date(date)
    plt.savefig(f'{surfaceTempActualPath}/SurfaceTempActual_{int(month)}-{int(day)}-{str(int(year))[2:]}.png')
    plt.close('all')


In [40]:
# import training dataset for depth checking
print("Importing training data....")
train_dataset = "practiceData3Years.nc"
ds = xarray.open_dataset(train_dataset)
df = ds.to_dataframe()

# create results directories if it doesn't exist
print("Creating directories....")
resultsPath = "ResultsTest"
thermoPath = resultsPath + "/ThermoclinePlots"
thermoActualPath = resultsPath + "/ThermoclineActualPlots"
surfaceTempActualPath = resultsPath + "/SurfaceTempActualMaps"
if not os.path.exists(resultsPath):
    os.makedirs(resultsPath)
    os.makedirs(thermoPath)
    os.makedirs(thermoActualPath)
    os.makedirs(surfaceTempActualPath)
    print("Directories Created Successfully")

# import ML model
print("Importing model....")
modelPath = "OTEC_miniBatchState.pth"
net = Net()
net.load_state_dict(torch.load(modelPath))
net.eval()

Importing training data....
Creating directories....
Directories Created Successfully
Importing model....


Net(
  (fc1): Linear(in_features=4, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=100, bias=True)
  (fc4): Linear(in_features=100, out_features=100, bias=True)
  (fc5): Linear(in_features=100, out_features=100, bias=True)
  (fc6): Linear(in_features=100, out_features=1, bias=True)
)

In [65]:
err = 0.5
lat = -32
long = 108
filtered_df = df.loc[(df['Latitude'].between(lat - err, lat + err)) 
        & (df['Longitude'].between(long - err, long + err)),['Depth']]
filtered_df.max()["Depth"]

1927.1

In [58]:
# main for loop to generate thermocline and get exergy for each time
print("Entering main loop....") 
for date in dateArray:
    # generate data for each location 
    print(f"Generating data for {date:.1f}....")
    for lat in latArray:
        for long in longArray:
            # land mask
            if not globe.is_land(lat, long):
                # get maximum depth of location from database
                maxDep = maxDepth(df, lat, long, areaIncr) 

                # generate thermocline for location using ML model
                print(f"Generating thermoclines for {lat:.1f}_{long:.1f}_{date:.1f}....") 
                tempDf = generateThermocline(lat, long, date, maxDep, depthIncr, net, thermoPath, True)
                tempDfActual = generateThermoclineActual(df, lat, long, thermoActualPath)
    
    # generateSurfaceTempActualMap(df, surfaceTempActualPath, date)

Entering main loop....
Generating data for 2455562.5....
Generating thermoclines for -32.0_108.0_2455562.5....
Generating data for 2455576.5....
Generating thermoclines for -32.0_108.0_2455576.5....
Generating data for 2455590.5....
Generating thermoclines for -32.0_108.0_2455590.5....
Generating data for 2455604.5....
Generating thermoclines for -32.0_108.0_2455604.5....
Generating data for 2455618.5....
Generating thermoclines for -32.0_108.0_2455618.5....
Generating data for 2455632.5....
Generating thermoclines for -32.0_108.0_2455632.5....
Generating data for 2455646.5....
Generating thermoclines for -32.0_108.0_2455646.5....
Generating data for 2455660.5....
Generating thermoclines for -32.0_108.0_2455660.5....
Generating data for 2455674.5....
Generating thermoclines for -32.0_108.0_2455674.5....
Generating data for 2455688.5....
Generating thermoclines for -32.0_108.0_2455688.5....
Generating data for 2455702.5....
Generating thermoclines for -32.0_108.0_2455702.5....
Generatin